# Fetch metadata first

In [ ]:
IDS = ['']

In [ ]:
import subprocess
import shlex
import shutil
import os 
import glob

import pandas as pd
import csv

In [ ]:
BASE_DIR = '/fastdata'
CONFIG_DIR = '/data/expression_atlas/configs'
OUTPUT_DIR = os.path.join(BASE_DIR, 'expression_atlas/runs/%s' % IDS[0])

STAR_REFERENCE_DIRECTORY = '/data/expression_atlas/genome/GRCh38.p14'

In [ ]:
with open(os.path.join(OUTPUT_DIR,'projects.txt'), 'w') as f:
    for i in IDS:
        f.write(i)
        f.write('\n')

In [ ]:
nxf_cmd_metadata = f"""sudo nextflow run nf-core/fetchngs \
--input {OUTPUT_DIR}/projects.txt \
--outdir {OUTPUT_DIR}/fetchngs_output \
--nf_core_pipeline rnaseq \
--skip_fastq_download \
-resume \
-profile docker \
-c {CONFIG_DIR}/fetchngs.config"""
print(nxf_cmd_metadata)

# Read Downloaded Metadata

In [ ]:
# samples = pd.read_csv('fetchngs_output/samplesheet/samplesheet.csv')
# samples.columns

## Filter samples

In [ ]:
# samples['library_strategy'].value_counts()

In [ ]:
# samples['library_layout'].value_counts()

In [ ]:
# samples['experiment_title'].value_counts()

In [ ]:
# samples['sample_description'].value_counts()

## Write filtered table to runsheet

In [ ]:
# samples['run_accession'].to_csv('runsheet.txt', index=False, header=False)

## Run custom download from sra bucket

In [ ]:
# Specify the path to downloaded metatdata below, for samplesheet.

nxf_cmd_fastq = f"""sudo nextflow run /home/jupyter-mitch/download_sra \
--samplesheet {OUTPUT_DIR}/fetchngs_output/metadata/{IDS[0]}.runinfo_ftp.tsv \
--output_directory {OUTPUT_DIR}/fetchngs_output/fastq \
-w {OUTPUT_DIR}/work \
-profile docker"""
print(nxf_cmd_fastq)

In [ ]:
# result = subprocess.run(shlex.split(nxf_cmd_fastq), capture_output=True, text=True)
# print(result.stdout)

# if result.returncode != 0:
#     print("Nextflow encountered an error:", result.stderr)
# else:
#     shutil.rmtree('./work')

# result_fastq = result.copy()

## Rearrange samplesheet fetch from fetchngs to make it compatible with nf-core rnaseq 

In [ ]:
samplesheet = pd.read_csv(os.path.join(OUTPUT_DIR, 'fetchngs_output', 'samplesheet/samplesheet.csv'))

samplesheet['fastq_1'] = [
                        os.path.join(OUTPUT_DIR, 'fetchngs_output', 'fastq/%s_%s.fastq.gz' % (e,r)) if s == 'SINGLE' else
                        os.path.join(OUTPUT_DIR, 'fetchngs_output', 'fastq/%s_%s_1.fastq.gz' % (e,r))
                        for e,r,s in zip(samplesheet['experiment_accession'], samplesheet['run_accession'], samplesheet['library_layout'])
                        ]
samplesheet['fastq_2'] = [
                        pd.NA if s == 'SINGLE' else
                        os.path.join(OUTPUT_DIR, 'fetchngs_output', 'fastq/%s_%s_2.fastq.gz' % (e,r))
                        for e,r,s in zip(samplesheet['experiment_accession'], samplesheet['run_accession'], samplesheet['library_layout'])
                        ]

samplesheet.to_csv(os.path.join(OUTPUT_DIR, 'fetchngs_output/samplesheet', 'samplesheet_alt.csv'), index=False, quoting=csv.QUOTE_NONNUMERIC)

samplesheet

# Fetchngs to actually download fastq files

In [ ]:
# nxf_cmd_fastq = f"""nextflow run nf-core/fetchngs \
# --input runsheet.txt \
# --outdir {OUTPUT_DIR} \
# --nf_core_pipeline rnaseq \
# -resume \
# -profile docker \
# -c {CONFIG_DIR}/fetchngs.config"""
# print(nxf_cmd_fastq)

In [ ]:
# result = subprocess.run(shlex.split(nxf_cmd_fastq), capture_output=True, text=True)
# print(result.stdout)

# if result.returncode != 0:
#     print("Nextflow encountered an error:", result.stderr)
# else:
#     shutil.rmtree('./work')

# result_fastq = result.copy()

In [ ]:
# # Read samplesheet from rnaseq
# rnaseq_samples = pd.read_csv('fetchngs_output/samplesheet/samplesheet.csv')

## Check that all files are correct and exist
Use MD5 checksums.

Function returns 1 if md5 matched correctly.  
Iterrate through and remove any samples with non-'1' values.

In [ ]:
# def check_fastq_integrity(ix, path, md5 = None):
#     if pathlib.Path(path).exists():
#         if not md5 is None:
#             checksum = subprocess.check_output(['md5sum', path]).decode().split(' ')[0]
#             if checksum != md5:
#                 return ix, 'md5sum error'
#             else:
#                 return ix, '1'
#     else:
#         return ix, 'fileExist error'
# # Parsing inputs for parallel starmap    
# inputs = {}
# for ix, row in rnaseq_samples.iterrows():
#     inputs[f'{ix}_a'] = [ix, row['fastq_1'], row['fastq_md5'].split(';')[0]]
#     if isinstance(row['fastq_2'], str):
#         inputs[f'{ix}_b'] = [ix, row['fastq_2'], row['fastq_md5'].split(';')[1]]
# with mp.Pool(NCPU) as ppool:
#     results = ppool.starmap(check_fastq_integrity, inputs.values())
    
# drop_indexes = []
# for r in results:
#     if r[1] != '1':
#         drop_indexes.append(r[0])

# rnaseq_samples.drop(drop_indexes, inplace=True)
# rnaseq_samples.to_csv('rnaseq_samplesheet.csv', index=False)

# Run nf-core/rnaseq pipeline

In [ ]:
nxf_cmd_align = f"""sudo nextflow run nf-core/rnaseq \
--input {OUTPUT_DIR}/fetchngs_output/samplesheet/samplesheet_alt.csv \
--fasta {STAR_REFERENCE_DIRECTORY}/rsem/GRCh38.p14.genome.fa \
--gtf {STAR_REFERENCE_DIRECTORY}/veliadb_v0c.gtf \
--star_index {STAR_REFERENCE_DIRECTORY}/star \
--transcript_fasta {STAR_REFERENCE_DIRECTORY}/rsem/genome.transcripts.fa \
--rsem_index {STAR_REFERENCE_DIRECTORY}/rsem \
--salmon_index {STAR_REFERENCE_DIRECTORY}/salmon \
--outdir {OUTPUT_DIR}/rnaseq_output \
-w {OUTPUT_DIR}/work \
-profile docker \
-c {CONFIG_DIR}/rnaseq.config \
-resume"""
print(nxf_cmd_align)

In [ ]:
# result = subprocess.run(shlex.split(nxf_cmd_align), capture_output=True, text=True)
# print(result.stdout)

# if result.returncode != 0:
#     print("Nextflow encountered an error:", result.stderr)
# else:
#     shutil.rmtree(f'{OUTPUT_DIR}/work')

# result_align = result.copy()